In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
#from fu import meaning_similarity,load_embeddings

In [ ]:
from vect import simi

In [ ]:
from models import captionning,classification,yolo,detection

In [ ]:
#embeddings=load_embeddings('vector.txt')

In [ ]:
def results(image):
    caption=captionning.show_n_generate(image)
    class1=classification.eval(image)
    class1=[tup[0] for tup in class1]
    class1=[x.replace('_',' ') for x in class1]
    class2=detection.get_prediction(image,0.5)[-1]
    class2=[x.replace('_',' ') for x in class2]
    class3=yolo.yolo(image)
    class3=[tup[0] for tup in class3]
    class3=[x.replace('_',' ') for x in class3]
    return caption,class1,class2,class3

In [ ]:
def compare_lists(list1, list2, list3):
    found_in_all = []
    found_in_two = []
    found_in_one = []

    for class1 in list1:
        found = False
        for class2 in list2:
            for class3 in list3:
                sim_score_1_2 = simi(class1, class2)
                sim_score_1_3 = simi(class1, class3)
                sim_score_2_3 = simi(class2, class3)

                if sim_score_1_2 > 0.8 and sim_score_1_3 > 0.8 and sim_score_2_3 > 0.8:
                    found_in_all.append(class1)
                    found = True
                    break
                elif (sim_score_1_2 > 0.8 and sim_score_1_3 > 0.8) or (sim_score_1_2 > 0.8 and sim_score_2_3 > 0.8) or (sim_score_1_3 > 0.8 and sim_score_2_3 > 0.8):
                    found_in_two.append(class1)
                    found = True
                    break

            if found:
                break

        if not found:
            found_in_one.append(class1)

    # Add classes from list2 and list3 not already categorized
    for class2 in list2:
        if class2 not in found_in_all and class2 not in found_in_two:
            found_in_one.append(class2)

    for class3 in list3:
        if class3 not in found_in_all and class3 not in found_in_two:
            found_in_one.append(class3)

    return found_in_all, found_in_two, found_in_one


In [ ]:
def final(found_in_all, found_in_two, found_in_one, caption):
    cap=caption.split()
    p1,p2,p3=0,0,0
    if found_in_all:
        for class_name in found_in_all:
            max_sim = 0
            for word in cap:
                sim = simi(class_name, word)
                if sim > max_sim:
                    max_sim = sim
            p1 += max_sim/len(found_in_all)

    if found_in_two:
        for class_name in found_in_two:
            max_sim = 0
            for word in cap:
                sim = simi(class_name, word)
                if sim > max_sim:
                    max_sim = sim
            p2 += max_sim/len(found_in_two)

    if found_in_one:
        for class_name in found_in_one:
            max_sim = 0
            for word in cap:
                sim = simi(class_name, word)
                if sim > max_sim:
                    max_sim = sim
            p3 += max_sim/len(found_in_one)
    
    if p1==0:
        if p2==0:
            total_score=p3
        elif p3==0:
            total_score=p2

    return caption,100*total_score




In [ ]:
imag='c:/Stage/Images/16.jpg'
image = Image.open(imag)
display(image)

In [ ]:
res=results(imag)
c=compare_lists(res[1],res[2],res[3])
final(c[0],c[1],c[2],res[0])

In [ ]:
res

In [ ]:
simi('mountain bike','bicycle')

In [ ]:
for i in range (53,82):
    imag=f'c:/Stage/Images/{i}.jpg'
    image = Image.open(imag)
    display(image)
    res=results(imag)
    c=compare_lists(res[1],res[2],res[3])
    print(final(c[0],c[1],c[2],res[0]))